# What is a Chain? (Very important foundation)
Chain = Connecting steps together

Think of a chain like a factory assembly line:
Input >> Step 1 >> Step 2 >> Step 3 >> Output

In LangChain:
Each step can be:
+ Prompt
+ LLM call
+ Tool
+ Retrieval

Chain helps you control the flow.

### LLMChain (Basic & Most Important)
What it is:
Single prompt + Single LLM call.

The building block of all other chains

Real-life analogy:
Asking one question to a teacher and getting one answer.

When to use:
+ Simple Q&A
+ Summarization
+ Translation
+ Text generation

Flow:
User Input >> Prompt Template >> LLM >> Output

Example:
prompt = "Explain {topic} in simple words"

LLMChain(prompt + LLM)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv() 

True

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

In [5]:
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple words."
)

In [6]:
chain = prompt | llm

# Prompt + LLM = Chain

In [7]:
response = chain.invoke({"topic": "LangChain"})
print(response.content)

LangChain is a framework designed to help developers build applications that use language models, like those from OpenAI. It makes it easier to create programs that can understand and generate human-like text. 

Think of LangChain as a toolkit that provides various tools and components to connect language models with other data sources, manage conversations, and handle tasks like searching for information or generating responses. This allows developers to create more complex and interactive applications that can chat, answer questions, or perform tasks based on natural language input.


### SimpleSequentialChain (Linear & Easy)

Here we pass Output of Step 1 as Input of Step 2
  
Real-life analogy:
Washing clothes >> Drying >> Folding

Each step depends on previous step.

When to use:
Straight forward workflows

Text >> refined text

Flow:
Input >> Chain A >> Chain B >> Final Output

Example:
Input text >> summarize >> translate
(Long Articles)  >>> input(Long article - 10 page) - Summarize - Output(Summarized Article - 1 page) >>>> input(Summarized Article - 1 page) - translate - Output(Translated Article - 1 page)



In [72]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [74]:
# Step 1: Summarize
summary_prompt = PromptTemplate.from_template(
    "Summarize the following text in one sentence:\n{text}"
)

# Step 2: Translate
translate_prompt = PromptTemplate.from_template(
    "Translate the following text into simple Kannada:\n{text}"
)

In [76]:
# Simple Sequential Chain (Runnable)
chain = summary_prompt | llm | translate_prompt | llm

In [78]:
# Input
input_text = """
LangChain is a framework designed to help developers build applications that use language models, like those from OpenAI. 
It makes it easier to create programs that can understand and generate human-like text. 

Think of LangChain as a toolkit that provides various tools and components to connect language models with other data sources, 
manage conversations, and handle tasks like searching for information or generating responses. This allows developers to create 
more complex and interactive applications that can chat, answer questions, or perform tasks based on natural language input.
"""

# Run
response = chain.invoke({"text": input_text})
print(response.content)

LangChain ಒಂದು ಫ್ರೇಮ್‌ವರ್ಕ್ ಆಗಿದ್ದು, ಅಭಿವೃದ್ಧಿಪಡಕರಿಗೆ ಭಾಷಾ ಮಾದರಿಗಳನ್ನು ಬಳಸಿಕೊಂಡು ಅಪ್ಲಿಕೇಶನ್‌ಗಳನ್ನು ನಿರ್ಮಿಸಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ. ಇದು ಡೇಟಾ ಮೂಲಗಳಿಗೆ ಸಂಪರ್ಕಿಸಲು, ಸಂಭಾಷಣೆಗಳನ್ನು ನಿರ್ವಹಿಸಲು ಮತ್ತು ನೈಸರ್ಗಿಕ ಭಾಷಾ ಇನ್ಪುಟ್ ಆಧಾರಿತ ಕಾರ್ಯಗಳನ್ನು ನಿರ್ವಹಿಸಲು ಉಪಕರಣಗಳನ್ನು ಒದಗಿಸುತ್ತದೆ.


### SequentialChain (with named intermediate outputs) 

In LangChain, SequentialChain is used for running components one after the other, where the output of one step is the input for the next, while RunnablePassthrough.assign is a method for adding new data fields to an input dictionary without altering the original input. 

Purpose: The SequentialChain class (or RunnableSequence in LangChain Expression Language, LCEL) allows you to combine multiple individual chains or runnables into a single, ordered workflow.

Functionality: The output generated by the first chain serves as the direct input to the second chain, and so on, until all chains in the sequence have executed.

Use Case: This is useful for multi-step tasks such as taking a topic from a user, generating a title from that topic with one chain, and then using that title to write a full story with a second chain.

RunnablePassthrough.assign

Purpose: RunnablePassthrough is a runnable that effectively returns its input as its output. The .assign() method extends this by allowing you to add new keys (and their corresponding values) to the input dictionary while still passing the original input data through the chain.

Functionality: It's a way to enrich the context of a pipeline. The values for the new keys can be static values or generated dynamically by other runnables or functions.

Use Case: A common use case is when you want to retrieve a context (e.g., from a vector store) and pass both the original user question and the retrieved context to an LLM chain.

RunnablePassthrough (CORE CONCEPT)
What is RunnablePassthrough?

RunnablePassthrough passes the input forward unchanged, while allowing you to add or enrich new data.

Simple definition:
“It keeps the original input safe while we add more information step by step.”

In [14]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [16]:
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in 3 bullet points:\n\n{text}"
)

In [17]:
keywords_prompt = ChatPromptTemplate.from_template(
    "Extract 5 important keywords from the text:\n\n{text}"
)

In [18]:
final_prompt = ChatPromptTemplate.from_template(
    """
Use the following information to give a clear explanation.

Original Text:
{text}

Summary:
{summary}

Keywords:
{keywords}

Final Explanation:
"""
)


In [19]:
summarize = summary_prompt | llm
keywords = keywords_prompt | llm
final = final_prompt | llm

In [20]:
def add_summary(x):
    response = summarize.invoke({"text": x["text"]})
    return response.content


def add_keywords(x):
    response = keywords.invoke({"text": x["text"]})
    return response.content


In [21]:
chain = (
    RunnablePassthrough()
    .assign(summary=add_summary)
    .assign(keywords=add_keywords)
    | final
)

In [22]:
input_text = """
LangChain is a framework that helps developers build applications using large language models.
It allows chaining prompts, models, tools, and retrievers together.
LangChain is widely used for chatbots, RAG systems, and AI agents.
"""

result = chain.invoke({"text": input_text})
print(result.content)

LangChain is a specialized framework that empowers developers to create applications that leverage large language models. This framework facilitates the integration of various components—such as prompts, models, tools, and retrievers—allowing them to be connected in a sequential manner, or "chained." As a result, LangChain is particularly popular for building applications like chatbots, retrieval-augmented generation (RAG) systems, and AI agents, making it a versatile tool in the realm of artificial intelligence development.


In [23]:
#Advanced example for the same type of chain

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

summarize = PromptTemplate.from_template(
    "Summarize in 1 line:\n{text}"
) | llm

keywords = PromptTemplate.from_template(
    "Extract 5 keywords from this:\n{text}"
) | llm

final = PromptTemplate.from_template(
    "Create a final beginner explanation using:\nSummary: {summary}\nKeywords: {keywords}"
) | llm

chain = (
    RunnablePassthrough()
    .assign(summary=lambda x: summarize.invoke({"text": x["text"]}).content)
    .assign(keywords=lambda x: keywords.invoke({"text": x["text"]}).content)
    | final
)

resp = chain.invoke({"text": "LangChain helps build LLM apps by chaining prompts, tools, and retrieval."})
print(resp.content)

**Understanding LangChain for Beginners**

LangChain is a powerful framework designed to help developers create applications that utilize Large Language Models (LLMs). At its core, LangChain simplifies the process of building these applications by integrating several key components: prompts, tools, and retrieval processes.

1. **LangChain**: This is the framework that brings everything together, making it easier to work with LLMs.
  
2. **LLM (Large Language Model)**: These are advanced AI models capable of understanding and generating human-like text. They can be used for various tasks, such as answering questions, writing content, or even having conversations.

3. **Apps**: With LangChain, developers can create applications that leverage the capabilities of LLMs. These apps can serve different purposes, from chatbots to content generators.

4. **Prompts**: In the context of LLMs, prompts are the instructions or questions given to the model to generate a response. LangChain helps in c

### RouterChain equivalent (choose Chain A or B)

Use RunnableBranch (like if/else routing).

RouterChain Equivalent using RunnableBranch

##### What is Routing in LangChain?
Routing means choosing one chain out of many based on the user input.
Just like if / else in programming. 

##### What is RunnableBranch?
RunnableBranch works like if / elif / else for chains.

It checks conditions one by one and runs the first matching chain.

Example: Customer Care System:

If question is billing → Billing team

If question is technical → Support team

Only one path is chosen

In [26]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

math_chain = PromptTemplate.from_template("Solve this math question: {q}") | llm
english_chain = PromptTemplate.from_template("Explain this in simple English: {q}") | llm

router = RunnableBranch(
    (RunnableLambda(lambda x: "math" in x["q"].lower()), math_chain),
    english_chain  # default goes here (NOT a tuple)
)

print(router.invoke({"q": "math: 12*8"}).content)
print(router.invoke({"q": "What is LangChain?"}).content)

The solution to the math question \( 12 \times 8 \) is \( 96 \).
LangChain is a tool that helps developers build applications using language models, like the ones that power chatbots or text generators. It makes it easier to connect these models with other data sources, manage conversations, and create more complex features. Essentially, LangChain helps you use language technology in your apps more effectively.


### RetrievalChain (RAG)

RetrievalChain is a modern RAG pattern where an LLM searches documents first and then answers using those documents, making responses accurate and reliable.

Large Language Models (LLMs) are powerful, but they have an important limitation: they do not automatically know your private or latest data. They are trained on public data up to a certain point in time and cannot directly read your PDFs, company documents, or databases. If you ask them questions outside their training knowledge, they may guess and produce incorrect answers, which is called hallucination.

To solve this problem, the industry uses a pattern called Retrieval-Augmented Generation (RAG). RetrievalChain is LangChain’s modern implementation of this pattern.

What RetrievalChain Means

RetrievalChain follows a very simple idea:

“First search for relevant documents, then use those documents to answer the question.”

Instead of expecting the LLM to know everything, we allow it to look up information at runtime and then generate an answer using that information.

In [1]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma

load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment/.env"

# 0) LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Knowledge base

docs = [
    Document(page_content="Venus is often called Earth's twin because of its similar size and proximity."),
    Document(page_content="Mars, known for its reddish appearance, is often referred to as the Red Planet."),
    Document(page_content="Jupiter, the largest planet in our solar system, has a prominent red spot."),
    Document(page_content="Saturn, famous for its rings, is sometimes mistaken for the Red Planet."),
]

# 2) Vector store (Chroma in-memory)
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = Chroma.from_documents(documents=docs, embedding=emb, collection_name="demo_rag")

retriever = vs.as_retriever(search_kwargs={"k": 2})

# 3) Prompt
qa_prompt = PromptTemplate.from_template(
    "Answer using ONLY this context:\n{context}\n\nQuestion: {question}\nAnswer:"
)

def format_docs(docs):
    return "\n".join(d.page_content for d in docs)

# 4) Retrieval + answer chain
def get_context(x):
    retrieved_docs = retriever.invoke(x["question"])
    return format_docs(retrieved_docs)

rag_chain = (
    {
        "context": get_context,
        "question": lambda x: x["question"],
    }
    | qa_prompt
    | llm
)

resp = rag_chain.invoke({"question": "Which plannet is know as Red planet?"})
print(resp.content)

Mars is known as the Red Planet.


In [53]:
# If needed (run once):
# !pip install -U langchain langchain-openai langchain-community chromadb python-dotenv

import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma

load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment/.env"

# 0) LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Knowledge base
docs = [
    Document(page_content="MySuits-shop sells suits and ethnic wear online and in-store."),
    Document(page_content="LangChain connects LLMs with prompts, tools, and retrieval (RAG)."),
]

# 2) Vector store (Chroma in-memory)
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = Chroma.from_documents(documents=docs, embedding=emb, collection_name="demo_rag")

retriever = vs.as_retriever(search_kwargs={"k": 2})

# 3) Prompt
qa_prompt = PromptTemplate.from_template(
    "Answer using ONLY this context:\n{context}\n\nQuestion: {question}\nAnswer:"
)

def format_docs(docs):
    return "\n".join(d.page_content for d in docs)

# 4) Retrieval + answer chain
def get_context(x):
    retrieved_docs = retriever.invoke(x["question"])
    return format_docs(retrieved_docs)

rag_chain = (
    {
        "context": get_context,
        "question": lambda x: x["question"],
    }
    | qa_prompt
    | llm
)

resp = rag_chain.invoke({"question": "What is LangChain used for?"})
print(resp.content)


LangChain is used for connecting LLMs with prompts, tools, and retrieval (RAG).


### Tool-using chain (LLM decides to call a tool)

LangChain supports tool calling via binding tools to the model (bind_tools).

Tool-Using Chain (LLM Decides to Call a Tool)

A tool-using chain is a pattern where the LLM is not limited to just text generation.
Instead, the LLM can decide to call an external tool (such as a calculator, API, database, or search function) when needed and then use the tool’s result to produce the final answer.

In simple words, the LLM becomes a decision maker:

If text knowledge is enough >> answer directly

If an action or real data is required >> call a tool

In [60]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    "Multiply two integers."
    return a * b

tool_llm = llm.bind_tools([multiply])

msg = tool_llm.invoke("What is 12 times 8? Use the tool.")
print("Model message:", msg)

# If the model called a tool, run it:
if getattr(msg, "tool_calls", None):
    call = msg.tool_calls[0]
    name = call["name"]
    args = call["args"]

    if name == "multiply":
        tool_result = multiply.invoke(args)
        print("Tool result:", tool_result)

        # Optional: ask LLM to form final answer using tool output
        final = llm.invoke(f"The tool returned {tool_result}. Reply with the final answer in one line.")
        print(final.content)
else:
    print("No tool call happened. Model answered directly:", msg.content)


Model message: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 56, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-Cn84rgqeLrhjHolnv5VGAsj6oN2Pt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b2361-99c2-7880-9e0d-06d8e76abb1d-0' tool_calls=[{'name': 'multiply', 'args': {'a': 12, 'b': 8}, 'id': 'call_dGqq156BLLKVgKOQYqQlisBQ', 'type': 'tool_call'}] usage_metadata={'input_tokens': 56, 'output_tokens': 17, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Tool result: 96
96
